In [ ]:
using Geant4
using Geant4.SystemOfUnits
#---Choose the Makie backend
#using GLMakie
#using WGLMakie
using CairoMakie
using GeometryBasics, Rotations, IGLWrap_jll  # to force loding G4Vis extension

In [ ]:
include(joinpath(@__DIR__, "../../../examples/basic/B2/DetectorB2a.jl"))

In [ ]:
#---Define Simulation Data struct------------------------------------------------------------------
mutable struct B2aSimData <: G4JLSimulationData
  tracks::Vector{Vector{Point3{Float64}}}   # vector of vector of step positions
end

In [ ]:
#---Step action------------------------------------------------------------------------------------
function stepaction(step::G4Step, app::G4JLApplication)::Nothing
  tracks = app.simdata[1].tracks
  p = GetPosition(GetPostStepPoint(step))
  push!(tracks[end], Point3{Float64}(x(p),y(p),z(p)))
  return
end
#---Tracking pre-action---------------------------------------------------------------------------- 
function pretrackaction(track::G4Track, app::G4JLApplication)::Nothing
  tracks = app.simdata[1].tracks
  p = GetPosition(track)[]
  push!(tracks, [Point3{Float64}(x(p),y(p),z(p))])
  return
end
#---begin-event-action---------------------------------------------------------------------------- 
function beginevent(::G4Event, app::G4JLApplication)::Nothing
  empty!(app.simdata[1].tracks)
  return
end

In [ ]:
#--------------------------------------------------------------------------------------------------
#---Particle Gun initialization--------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------
particlegun = G4JLGunGenerator(particle = "proton", 
                               energy = 3GeV, 
                               direction = G4ThreeVector(0,0,1), 
                               position = G4ThreeVector(0,0,-2940.0))


In [ ]:
#---Create the Application-------------------------------------------------------------------------
app = G4JLApplication(detector=B2aDetector(nChambers=5),          # detector with parameters
                      simdata = B2aSimData([]),                   # simulation data structure
                      generator = particlegun,                    # particle gun generator
                      physics_type = FTFP_BERT,                   # what physics list to instantiate
                      stepaction_method = stepaction,             # step action method
                      begineventaction_method = beginevent,       # begin-event action (initialize per-event data)
                      pretrackaction_method = pretrackaction)     # pre-tracking action
              
#---Configure, Initialize and Run------------------------------------------------------------------                      
configure(app)
initialize(app)
beamOn(app, 8)

In [ ]:
world = GetWorldVolume()
tracks = app.simdata[1].tracks;

In [ ]:
beamOn(app, 1)

fig = Figure(resolution=(1024,1024))
s = LScene(fig[1,1])
Geant4.draw!(s, world)
for t in tracks
    lines!(t)
end
display(fig)